<a href="https://colab.research.google.com/github/sousam/Clei-Laclo-2018/blob/master/New_Autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [57]:
import tensorflow as tf
from tensorflow.keras import datasets
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv1D,AveragePooling1D,Flatten,Dense
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import ReLU
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.utils import plot_model
import numpy as np
import pandas as pd
import sklearn
from sklearn import preprocessing, decomposition, model_selection
#tf.compat.v1.logging.set_verbosity(tf.logging.ERROR)

In [58]:
#loading the data
data = pd.read_table('australian.dat', sep="\s+", header=None).dropna()
#data = pd.read_csv('australian-encoded.csv',header=0)
print(data)
x = data[data.columns[:-1]].to_numpy()
y = data[data.columns[-1]].to_numpy()

x.reshape(x.shape[0],x.shape[1],1)
  
#Scaling
scaler = preprocessing.MinMaxScaler()
    
scaled_data = scaler.fit_transform(x)
  
#75% percent of dataset is designated to training
X_train, X_test = model_selection.train_test_split(scaled_data, test_size=0.2)

     0      1       2   3   4   5      6   7   8   9   10  11   12    13  14
0     1  22.08  11.460   2   4   4  1.585   0   0   0   1   2  100  1213   0
1     0  22.67   7.000   2   8   4  0.165   0   0   0   0   2  160     1   0
2     0  29.58   1.750   1   4   4  1.250   0   0   0   1   2  280     1   0
3     0  21.67  11.500   1   5   3  0.000   1   1  11   1   2    0     1   1
4     1  20.17   8.170   2   6   4  1.960   1   1  14   0   2   60   159   1
..   ..    ...     ...  ..  ..  ..    ...  ..  ..  ..  ..  ..  ...   ...  ..
685   1  31.57  10.500   2  14   4  6.500   1   0   0   0   2    0     1   1
686   1  20.67   0.415   2   8   4  0.125   0   0   0   0   2    0    45   0
687   0  18.83   9.540   2   6   4  0.085   1   0   0   0   2  100     1   1
688   0  27.42  14.500   2  14   8  3.085   1   1   1   0   2  120    12   1
689   1  41.00   0.040   2  10   4  0.040   0   1   1   0   1  560     1   1

[690 rows x 15 columns]


In [59]:
epochs = 100
batch_size = len(X_test)

In [60]:
n_inputs = len(X_train[0])
# define encoder
visible = Input(shape=(n_inputs,))
# encoder level 1
e = Dense(n_inputs*2)(visible)
#e = BatchNormalization()(e)
e = ReLU()(e)
# encoder level 2
e = Dense(n_inputs)(e)
#e = BatchNormalization()(e)
e = ReLU()(e)
# bottleneck
n_bottleneck = 10
bottleneck = Dense(n_bottleneck, activation='sigmoid')(e)
# define decoder, level 1
d = Dense(n_inputs)(bottleneck)
#d = BatchNormalization()(d)
d = ReLU()(d)
# decoder level 2
d = Dense(n_inputs*2)(d)
#d = BatchNormalization()(d)
d = ReLU()(d)
# output layer
output = Dense(n_inputs, activation='linear')(d)
# define autoencoder model
model = Model(inputs=visible, outputs=output)

#vector_loss = tf.losses.mean_squared_error(visible, output, reduction=tf.losses.Reduction.NONE)

In [61]:
opt = tf.keras.optimizers.SGD()

vector_loss = tf.keras.losses.MeanSquaredError(reduction=tf.losses.Reduction.NONE)

In [62]:
model.compile(optimizer=opt, loss=vector_loss)

In [63]:
history = model.fit(X_train, X_train, batch_size=batch_size, epochs=epochs, verbose=2, validation_data=(X_test,X_test))

Train on 552 samples, validate on 138 samples
Epoch 1/100
552/552 - 1s - loss: 0.1511 - val_loss: 0.0940
Epoch 2/100
552/552 - 0s - loss: 0.0920 - val_loss: 0.0904
Epoch 3/100
552/552 - 0s - loss: 0.0911 - val_loss: 0.0902
Epoch 4/100
552/552 - 0s - loss: 0.0908 - val_loss: 0.0893
Epoch 5/100
552/552 - 0s - loss: 0.0909 - val_loss: 0.0890
Epoch 6/100
552/552 - 0s - loss: 0.0909 - val_loss: 0.0895
Epoch 7/100
552/552 - 0s - loss: 0.0907 - val_loss: 0.0898
Epoch 8/100
552/552 - 0s - loss: 0.0906 - val_loss: 0.0895
Epoch 9/100
552/552 - 0s - loss: 0.0907 - val_loss: 0.0900
Epoch 10/100
552/552 - 0s - loss: 0.0906 - val_loss: 0.0899
Epoch 11/100
552/552 - 0s - loss: 0.0909 - val_loss: 0.0894
Epoch 12/100
552/552 - 0s - loss: 0.0905 - val_loss: 0.0900
Epoch 13/100
552/552 - 0s - loss: 0.0907 - val_loss: 0.0904
Epoch 14/100
552/552 - 0s - loss: 0.0905 - val_loss: 0.0899
Epoch 15/100
552/552 - 0s - loss: 0.0904 - val_loss: 0.0886
Epoch 16/100
552/552 - 0s - loss: 0.0904 - val_loss: 0.0900
Epo

In [64]:
encoder = Model(inputs=visible, outputs=bottleneck)

X_encode = encoder.predict(x)

encodedData = pd.DataFrame(data=X_encode)

#Adding a label to the encoded dataframe
encodedData['label'] = y

In [65]:
print(encodedData)

encodedData.to_csv('new-australian-encoded.csv',index=False)

            0         1         2  ...         8         9  label
0    1.000000  1.000000  0.000000  ...  0.000000  0.000000      0
1    0.999987  0.203034  0.999631  ...  0.999359  0.006312      0
2    1.000000  0.010035  0.999998  ...  0.999997  0.000038      0
3    1.000000  0.998759  0.031508  ...  0.999717  0.000004      1
4    1.000000  1.000000  0.000000  ...  0.000315  0.000000      1
..        ...       ...       ...  ...       ...       ...    ...
685  1.000000  0.999545  0.179186  ...  0.998400  0.000064      1
686  1.000000  0.999866  0.000116  ...  0.000007  0.001357      0
687  0.998341  0.341659  0.996334  ...  0.995204  0.065052      1
688  0.999976  0.965075  0.995087  ...  0.995371  0.003991      1
689  1.000000  0.000018  1.000000  ...  1.000000  0.000000      1

[690 rows x 11 columns]


In [66]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

scaler2 = preprocessing.MinMaxScaler()
    
encodedData.iloc[:,:-1] = scaler2.fit_transform(encodedData.iloc[:,:-1])

X_train, X_test, y_train, y_test = train_test_split(encodedData.iloc[:,:-1], encodedData.iloc[:,-1], test_size=0.33, random_state=1)
model = MLPClassifier()
# fit the model on the training set
model.fit(X_train, y_train)
# make predictions on the test set
yhat = model.predict(X_test)
# calculate classification accuracy
acc = accuracy_score(y_test, yhat)
print(acc)

0.7105263157894737


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
